In [1]:
import pandas as pd
data_path = "/home/tofan/data1/csv/LSTM/data_v2/combined_data_all.csv"
data = pd.read_csv(data_path, nrows=1000)

In [2]:
pd.set_option('display.max_columns', None)
# Вывод обработанного датафрейма
data.head(20)

,Unnamed: 0,TypeBuyIn,TournamentNumber,StartDateUtc,Hand,Level,Street_id,Pot,Card1,Card2,Card3,Card4,Card5,Dealer,Seat,Round,ActionOrder,PlayerId,PlayerName,Stack,SPR,Action,Bet,Allin,Showdown_1,Showdown_2,PlayerWins,WinAmount,Position
0,246,MTT 1000,607464523,2024-02-05T16:13:26Z,1958559965,2500.0,1,19600.0,9h,7h,2s,NaN,NaN,0,4,2,2,3,morfoza,659750.0,33.66,Bet,9800.0,0,NaN,NaN,NaN,NaN,MP1
1,795,MTT 1000,607464523,2024-02-05T16:32:54Z,1958593531,3000.0,0,16700.0,NaN,NaN,NaN,NaN,NaN,1,7,1,6,6,bagzik,212350.0,12.72,Fold,0.0,0,NaN,NaN,NaN,NaN,BTN
2,611,MTT 1000,607464523,2024-02-05T16:26:42Z,1958582651,3000.0,2,113112.0,7s,Jh,2c,Qd,NaN,0,4,4,2,3,morfoza,837006.0,7.40,Check,0.0,0,NaN,NaN,NaN,NaN,MP2
3,243,MTT 1000,607464523,2024-02-05T16:13:26Z,1958559965,2500.0,0,15850.0,NaN,NaN,NaN,NaN,NaN,0,2,1,6,1,widregkzn,244650.0,15.44,Call,3750.0,0,NaN,NaN,NaN,NaN,SB
4,496,MTT 1000,607464523,2024-02-05T16:22:18Z,1958574984,2500.0,2,19900.0,Qs,Tc,7c,7s,NaN,0,4,3,1,3,morfoza,856898.0,43.06,Bet,9950.0,0,NaN,NaN,NaN,NaN,SB
5,327,MTT 1000,607464523,2024-02-05T16:15:52Z,1958564151,2500.0,1,12768.0,7d,4s,8d,NaN,NaN,0,2,2,3,1,widregkzn,206013.0,16.14,Fold,0.0,0,NaN,NaN,NaN,NaN,MP3
6,359,MTT 1000,607464523,2024-02-05T16:16:29Z,1958565177,2500.0,3,107437.0,9h,9d,8s,Jd,2s,0,4,5,3,3,morfoza,700349.0,6.52,Call,35812.0,0,NaN,NaN,NaN,NaN,MP3
7,454,MTT 1000,607464523,2024-02-05T16:20:37Z,1958572126,2500.0,0,6150.0,NaN,NaN,NaN,NaN,NaN,0,6,1,2,5,tbiheiipab,308350.0,50.14,Fold,0.0,0,NaN,NaN,NaN,NaN,MP1
8,465,MTT 1000,607464523,2024-02-05T16:20:37Z,1958572126,2500.0,2,12400.0,3s,2h,8d,7d,NaN,0,3,3,1,2,shavasana,459250.0,37.04,Check,0.0,0,NaN,NaN,NaN,NaN,SB
9,551,MTT 1000,607464523,2024-02-05T16:24:04Z,1958578012,3000.0,2,22800.0,Kd,Kc,Qs,3c,NaN,0,2,4,3,1,widregkzn,154426.0,6.77,Fold,0.0,0,NaN,NaN,NaN,NaN,MP2


In [3]:
print(data.columns.to_list())


['Unnamed: 0', 'TypeBuyIn', 'TournamentNumber', 'StartDateUtc', 'Hand', 'Level', 'Street_id', 'Pot', 'Card1', 'Card2', 'Card3', 'Card4', 'Card5', 'Dealer', 'Seat', 'Round', 'ActionOrder', 'PlayerId', 'PlayerName', 'Stack', 'SPR', 'Action', 'Bet', 'Allin', 'Showdown_1', 'Showdown_2', 'PlayerWins', 'WinAmount', 'Position']


In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

def create_features(data):
    """
    Создаёт дополнительные признаки для пайплайна:
    
    Базовые признаки:
      - UniquePlayerId: уникальный ID для каждого игрока
      - ActionHistory, ActionOrder, ActionCount
      - RaiserPosNum: позиция последнего, кто сделал Bet или Raise
      - RelativePosition: NoRaiser / Self / Before / After
      - PotOdds = Bet / (Pot + Bet)
      - PreviousBetSize (shift(1) внутри раздачи)
      - SPR = Stack / Pot
      - StackCategory: Short / Mid / Deep
      - BoardTexture: Monotone / Rainbow / Neutral
      - BoardDangerLevel: High / Low / Medium
    
    Исторические признаки:
      - Статистика действий по позициям (частота каждого действия)
      - VPIP (процент добровольного входа в банк)
      - PFR (процент рейза на префлопе)
      - Факторы агрессии на разных улицах
      - Статистика ставок (средняя, стд, мин, макс)
      - Тенденции к продолжению агрессии
      - Адаптация размеров ставок
      - Статистика выигрышей по позициям
    """
    # Создаем копию датафрейма
    data = data.copy()
    
    # Создаем уникальный ID для каждого игрока нlearning_rate
        "MP2": 3,
        "MP3": 4,
        "CO": 5,
        "BTN": 6,
        "SB": 7,
        "BB": 8
    }

    # ============= БАЗОВЫЕ ПРИЗНАКИ =============
    
    # 1) История действий в рамках раздачи
    data["ActionHistory"] = (
        data.groupby("Hand")["Action"]
            .transform(lambda x: x.shift(1).fillna("").astype(str).cumsum())
    )
    data["ActionCount"] = data.groupby("Hand")["Action"].transform("count")
    data["ActionOrder"] = data.groupby("Hand").cumcount()

    # 2) Преобразование позиций в числовые значения
    data["PositionNum"] = data["Position"].map(lambda x: pos_map.get(x, -1))

    # 3) Находим позицию последнего беттора/рейзера
    raiser_mask = data['Action'].isin(['Bet', 'Raise'])
    data['TempRaiserPos'] = np.where(raiser_mask, data['PositionNum'], np.nan)
    data["RaiserPosNum"] = data.groupby("Hand")['TempRaiserPos'].transform(
        lambda x: x.ffill().fillna(-1)
    )
    data = data.drop('TempRaiserPos', axis=1)

    # 4) Определяем RelativePosition
    data["RelativePosition"] = np.where(
        data["RaiserPosNum"] == -1,
        "NoRaiser",
        np.where(
            data["PositionNum"] == data["RaiserPosNum"],
            "Self",
            np.where(
                data["PositionNum"] < data["RaiserPosNum"],
                "Before",
                "After"
            )
        )
    )

    # 5) Пот-оддсы с безопасным делением
    data["PotOdds"] = np.where(
        data["Pot"] + data["Bet"] > 0,
        data["Bet"] / (data["Pot"] + data["Bet"]),
        0
    )

    # 6) Предыдущий размер ставки
    data["PreviousBetSize"] = data.groupby("Hand")["Bet"].transform(lambda x: x.shift(1).fillna(0))

    # 7) SPR с безопасным делением
    data["SPR"] = np.where(
        data["Pot"] > 0,
        data["Stack"] / data["Pot"],
        0
    )

    # 8) Категоризация стека
    data["StackCategory"] = data["Stack"].apply(categorize_stack)

    # 9) Анализ текстуры борда
    data["BoardTexture"] = data.apply(get_board_texture, axis=1)
    data["BoardDangerLevel"] = data["BoardTexture"].map({
        "Monotone": "High",
        "Rainbow": "Low",
        "Neutral": "Medium"
    })

    # ============= ИСТОРИЧЕСКИЕ ПРИЗНАКИ =============

    # 1) Статистика действий по позициям
    for action in ['Bet', 'Raise', 'Call', 'Fold']:
        # Создаем маску для действия
        action_mask = data['Action'] == action
        # Считаем частоту действия для каждой комбинации игрок-позиция
        action_counts = data[action_mask].groupby(['UniquePlayerId', 'Position']).size()
        total_counts = data.groupby(['UniquePlayerId', 'Position']).size()
        action_freq = (action_counts / total_counts * 100).round(2)
        
        # Преобразуем в DataFrame для merge
        freq_df = action_freq.reset_index()
        freq_df.columns = ['UniquePlayerId', 'Position', f'{action}Frequency']
        data = data.merge(freq_df, on=['UniquePlayerId', 'Position'], how='left')

    # 2) VPIP и PFR
    preflop_data = data[data['Street_id'] == 0]
    preflop_hands = preflop_data.groupby('UniquePlayerId')['Hand'].nunique()
    
    # VPIP
    vpip_hands = preflop_data[
        preflop_data['Action'].isin(['Bet', 'Raise', 'Call'])
    ].groupby('UniquePlayerId')['Hand'].nunique()
    vpip = (vpip_hands / preflop_hands * 100).round(2)
    data['VPIP'] = data['UniquePlayerId'].map(vpip.fillna(0))
    
    # PFR
    pfr_hands = preflop_data[
        preflop_data['Action'].isin(['Bet', 'Raise'])
    ].groupby('UniquePlayerId')['Hand'].nunique()
    pfr = (pfr_hands / preflop_hands * 100).round(2)
    data['PFR'] = data['UniquePlayerId'].map(pfr.fillna(0))

    # 3) Агрессия по улицам
    for street in [0, 1, 2, 3]:  # preflop, flop, turn, river
        street_data = data[data['Street_id'] == street]
        
        # Считаем агрессивные и пассивные действия
        aggressive_mask = street_data['Action'].isin(['Bet', 'Raise'])
        passive_mask = street_data['Action'].isin(['Call', 'Check'])
        
        aggressive_counts = aggressive_mask.groupby(street_data['UniquePlayerId']).sum()
        passive_counts = passive_mask.groupby(street_data['UniquePlayerId']).sum()
        
        # Вычисляем фактор агрессии
        aggression = (aggressive_counts + 0.01) / (passive_counts + 0.01)
        data[f'AggressionFactor_Street{street}'] = data['UniquePlayerId'].map(aggression)

    # 4) Статистика ставок
    bet_data = data[data['Action'].isin(['Bet', 'Raise'])]
    bet_stats = bet_data.groupby('UniquePlayerId')['Bet'].agg([
        ('mean', 'mean'),
        ('std', 'std'),
        ('min', 'min'),
        ('max', 'max')
    ]).round(2).fillna(0)
    
    for stat in ['mean', 'std', 'min', 'max']:
        data[f'BetSize_{stat}'] = data['UniquePlayerId'].map(bet_stats[stat])

    # 5) Тенденция к продолжению агрессии
    data = data.sort_values(['Hand', 'ActionOrder'])
    aggressive_mask = data['Action'].isin(['Bet', 'Raise'])
    data['PrevAggressive'] = aggressive_mask.groupby(data['Hand']).transform(lambda x: x.shift(1).fillna(False))
    
    # Считаем продолжение агрессии
    continuation_mask = aggressive_mask & data['PrevAggressive']
    continuation_sum = continuation_mask.groupby([data['UniquePlayerId'], data['Hand']]).transform('sum')
    opportunity_sum = data['PrevAggressive'].groupby([data['UniquePlayerId'], data['Hand']]).transform('sum')
    
    data['ContinuationTendency'] = (continuation_sum / (opportunity_sum + 0.01)).groupby(data['UniquePlayerId']).transform('mean')
    data = data.drop('PrevAggressive', axis=1)

    # 6) Адаптация к оппонентам (изменчивость размеров ставок)
    bet_mask = data['Action'].isin(['Bet', 'Raise'])
    
    # Вычисляем стандартное отклонение и среднее ставок
    std_bets = data[bet_mask].groupby(['UniquePlayerId', 'Hand'])['Bet'].transform('std')
    mean_bets = data[bet_mask].groupby(['UniquePlayerId', 'Hand'])['Bet'].transform('mean')
    
    # Вычисляем коэффициент вариации
    data.loc[bet_mask, 'TempCV'] = std_bets / (mean_bets + 0.01)
    player_avg_cv = data.groupby('UniquePlayerId')['TempCV'].transform('mean')
    data['BetAdaptation'] = data['TempCV'].fillna(player_avg_cv)
    data = data.drop('TempCV', axis=1)

    # 7) Статистика выигрышей по позициям
    win_sum = data.groupby(['UniquePlayerId', 'Position'])['PlayerWins'].transform('sum')
    hand_count = data.groupby(['UniquePlayerId', 'Position']).transform('size')
    data['WinRate'] = (win_sum / hand_count * 100).round(2)

    return data

def categorize_stack(stack):
    """Категоризация размера стека."""
    if stack < 10:
        return "Short"
    elif stack < 30:
        return "Mid"
    return "Deep"

def get_board_texture(row):
    """
    Анализ текстуры борда на основе мастей карт.
    Учитываем только непустые карты (Card1..Card5).
    Масть определяется как последний символ ('9h' -> 'h').
    """
    suits = []
    for col in ["Card1", "Card2", "Card3", "Card4", "Card5"]:
        card = row[col]
        if pd.notna(card) and isinstance(card, str) and card != "":
            suits.append(card[-1].lower())
    
    if len(suits) == 0:
        return "Neutral"

    unique_suits = set(suits)
    if len(unique_suits) == 1:
        return "Monotone"
    if len(unique_suits) == len(suits):
        return "Rainbow"
    return "Neutral"

def save_processed_dataset(processed_data, original_path):
    """
    Сохраняет обработанный датасет с новыми признаками.
    
    Args:
        processed_data (pd.DataFrame): Обработанный датасет с новыми признаками
        original_path (str): Путь к исходному файлу
    """
    # Получаем директорию и имя исходного файла
    directory = os.path.dirname(original_path)
    filename = os.path.basename(original_path)
    name, ext = os.path.splitext(filename)
    
    # Создаем директорию для обработанных данных, если её нет
    processed_dir = os.path.join(directory, 'processed_data')
    os.makedirs(processed_dir, exist_ok=True)
    
    # Добавляем timestamp к имени файла
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    new_filename = f"{name}_processed_{timestamp}{ext}"
    output_path = os.path.join(processed_dir, new_filename)
    
    # Сохраняем датасет
    processed_data.to_csv(output_path, index=False)
    print(f"Датасет сохранен в: {output_path}")
    
    # Создаем файл с описанием признаков
    features_filename = f"{name}_features_{timestamp}.txt"
    features_path = os.path.join(processed_dir, features_filename)
    
    new_features = [col for col in processed_data.columns if col not in data.columns]
    
    with open(features_path, 'w', encoding='utf-8') as f:
        f.write("Новые признаки в датасете:\n\n")
        for feature in new_features:
            f.write(f"- {feature}\n")
            
        # Добавляем базовую статистику
        f.write("\nБазовая статистика:\n")
        f.write(f"Всего строк: {len(processed_data)}\n")
        f.write(f"Всего признаков: {len(processed_data.columns)}\n")
        f.write(f"Количество уникальных игроков: {processed_data['UniquePlayerId'].nunique()}\n")
        
    print(f"Описание признаков сохранено в: {features_path}")

# Пример использования
if __name__ == "__main__":
    # Путь к исходному файлу
    data_path = '/home/tofan/data1/csv/LSTM/data_v2/combined_data_all.csv'
    
    # Читаем данные
    data = pd.read_csv(data_path, nrows=None)
    
    # Применяем создание признаков
    processed_data = create_features(data)
    
    # Сохраняем результаты
    save_processed_dataset(processed_data, data_path)

In [ ]:
import pandas as pd

# Путь к файлу
file_path = "/home/tofan/data1/csv/LSTM/data_v2/processed_data/combined_data_all_processed_20250223_185211.csv"

# Загружаем CSV
df = pd.read_csv(file_path)

# Определяем допустимые значения Action
valid_actions = {"Fold", "Bet", "Check", "Call", "Raise"}

# Фильтруем только нужные Action
df_filtered = df[df["Action"].isin(valid_actions)]

# Выводим статистику
print(f"Исходное количество строк: {len(df)}")
print(f"Оставлено строк после фильтрации: {len(df_filtered)}")

# Сохраняем обработанный датасет
output_path = "/home/tofan/data1/csv/LSTM/data_v2/processed_data/filtered_data.csv"
df_filtered.to_csv(output_path, index=False)

print(f"Обработанные данные сохранены в: {output_path}")
